In [46]:
import stat
import pandas as pd
import numpy as np
import requests 
import json
import datetime

from bs4 import BeautifulSoup

appKey = 'l7xx00cbac0e4ef64360ab73928c9f175219'
secretKey = 'c961e0f624dc41378089daffaa3c1d4c'
access_url = 'https://gate.nicednb.com/nice/oauth/v1.0/accesstoken'
cleanmatch_url = 'https://gate.nicednb.com/global/directplus/v1.0/identify/match/cleansematch'
datablock_url = 'https://gate.nicednb.com/global/directplus/v1.0/enrich/dproduct/datablock/'

search_param = {
    '후보갯수': 5,
    '기준점': 5
}

In [12]:
url = access_url
headers = {
    'Content-Type': 'application/json;charset=utf-8'
}
body = {
    "appKey": appKey,
    "appSecret": secretKey,
    "grantType": "client_credentials",
    "scope":"oob"
}

res = requests.post(url, headers=headers, json=body)
token_body = res.json()

begin_time = datetime.datetime.now()
end_time = begin_time + datetime.timedelta(seconds=token_body['expiresIn'])

token_body['end_time'] = end_time

accesstoken = token_body
print(token_body)

{'accessToken': 'da17a4bf-ff2e-4f07-a7b1-c444ea0bd1a3', 'tokenType': 'Bearer', 'expiresIn': 86400, 'scope': 'oob', 'end_time': datetime.datetime(2023, 8, 26, 11, 24, 41, 499328)}


In [73]:
search_list = {
    # '법인1':{
    #     '등록번호':'',
    #     '회사명':'삼성전자',
    #     '소재국가':'KR',
    #     '주소':'경기 수원시 영통구',
    # },
    '법인2':{
        '등록번호':'',
        '회사명':'애플',
        '소재국가':'KR',
        '주소':'서울특별시 강남구 영동대로 517'          
    },
    '법인3':{
        '등록번호':'',
        '회사명':'Hitachi Data Systems Pte Ltd',
        '소재국가':'SG',
        '주소':'18 Cross Street, #10-08 China Square Central Singapore',
        # '주소':'18 Cross Street',
    },
    '법인4':{
        '등록번호':'',
        '회사명':'Beijing Dongjin Trading co. ,LTD',
        '소재국가':'CN',
        '주소':'No.35 Dongbahe, Chaoyang District, Beijing',
        # '주소':'No.35 Dongbahe, Chaoyang District',            
    },
    '법인5':{
        '등록번호':'',
        '회사명':'BRUSH(HONG KONG) LTD.',
        '소재국가':'HK',
        '주소':'1008, Cornell Centre, 50 Wing Tai Road, Chai Wan',
        # '주소':'1008, Cornell Centre, 50 Wing Tai Road',
    },        
    '법인6':{
        '등록번호':'',
        '회사명':'FPT SOFTWARE COMANY LIMITED, DA NANG BRANCH',
        '소재국가':'VN',
        '주소':'FPT Building, Road No.1, Danang Industrial Zone An Don, An Hai Bac ward, Son Tra District, Da Nang City',
        # '주소':'FPT Building, Road No.1, Danang Industrial Zone An Don, An Hai Bac ward, Son Tra District',                        
    }, 
}

search_list_keys = list(search_list.keys())

In [74]:
c_num = '법인2'

if datetime.datetime.now() >= accesstoken['end_time']:

    accesstoken = get_access_token()
    headers['Authorization'] = 'Bearer ' + accesstoken['accessToken']

body = {
    'registrationNumber': search_list[c_num]['등록번호'],
    'name': search_list[c_num]['회사명'],
    'streetAddressLine1': search_list[c_num]['주소'],
    'countryISOAlpha2Code': search_list[c_num]['소재국가'],
    'candidateMaximumQuantity': search_param['후보갯수'],
    'confidenceLowerLevelThresholdValue': search_param['기준점'],
}

# 한국어 검색은 언어 parameter 수정 필요 - 한국어로(디폴트:en-US)
if body['countryISOAlpha2Code']=='KR':
    body['inLanguage'] = 'ko-hang-KR'

# Test중이므로 Call수를 아끼려면 search_results는 따로 저장해놓고 꺼내쓰도록 해야할 듯
res = requests.get(url, headers=headers, params=body)
search_results = res.json()

search_results

{'transactionDetail': {'transactionID': 'rrt-02582b39f33ee77e2-a-se-17848-1357421-1',
  'transactionTimestamp': '2023-08-25T09:16:55.961Z',
  'inLanguage': 'ko-hang-KR',
  'serviceVersion': '1'},
 'inquiryDetail': {'inLanguage': 'ko-hang-KR',
  'registrationNumber': '',
  'name': '애플',
  'address': {'countryISOAlpha2Code': 'KR',
   'streetAddressLine': {'line1': '서울특별시 강남구 영동대로 517'}},
  'candidateMaximumQuantity': 5,
  'confidenceLowerLevelThresholdValue': 5},
 'candidatesMatchedQuantity': 1,
 'matchDataCriteria': 'Name and Address Lookup',
 'matchCandidates': [{'displaySequence': 1,
   'organization': {'duns': '689085396',
    'websiteAddress': [],
    'dunsControlStatus': {'operatingStatus': {'description': 'Out of business',
      'dnbCode': 403},
     'isMailUndeliverable': None},
    'primaryName': '애플',
    'tradeStyleNames': [],
    'telephone': [{'telephoneNumber': '025641236', 'isUnreachable': None}],
    'primaryAddress': {'addressCountry': {'isoAlpha2Code': 'KR',
      'nam

In [71]:
for c_num in search_list.keys():

    print(search_list [c_num]['회사명'])

    # 토큰시간 만료되었으면 재인증, 아니면 진행
    if datetime.datetime.now() >= accesstoken['end_time']:

        accesstoken = get_access_token()
        headers['Authorization'] = 'Bearer ' + accesstoken['accessToken']

    body = {
        'registrationNumber': search_list[c_num]['등록번호'],
        'name': search_list[c_num]['회사명'],
        'streetAddressLine1': search_list[c_num]['주소'],
        'countryISOAlpha2Code': search_list[c_num]['소재국가'],
        'candidateMaximumQuantity': search_param['후보갯수'],
        'confidenceLowerLevelThresholdValue': search_param['기준점'],
    }

    # 한국어 검색은 언어 parameter 수정 필요 - 한국어로(디폴트:en-US)
    if body['countryISOAlpha2Code']=='KR':
        body['inLanguage'] = 'ko-hang-KR'

    # Test중이므로 Call수를 아끼려면 search_results는 따로 저장해놓고 꺼내쓰도록 해야할 듯
    res = requests.get(url, headers=headers, params=body)
    search_results = res.json()
    
    # 일단은 1번으로 나오는 결과물을 뽑도록 해 놨으나, 정확도 보완차원에서 결과물 다 검토하긴 해야함
    try:

        for i in range(len(search_results['matchCandidates'])):
        
            best_results = search_results['matchCandidates'][i]

            duns = best_results['organization']['duns']
            companyname = best_results['organization']['primaryName']
            address = best_results['organization']['primaryAddress']
            regi_number = best_results['organization']['registrationNumbers']
            matchscore = best_results['matchQualityInformation']['confidenceCode']
            
            datablock_search[c_num] = {
                '구분': c_num,
                '회사명': companyname,
                '주소': address,
                '등록번호': regi_number,
                'score': matchscore,
                'duns': duns,
            }
            
            print(datablock_search)[c_num]
            
            p_or_f = input('OK or NEXT - Input "ok" or "next": ')

            if p_or_f == "ok":

                datablock_search[c_num] = {
                    '구분': c_num,
                    '회사명': companyname,
                    '주소': address,
                    '등록번호': regi_number,
                    'score': matchscore,
                    'duns': duns,
                }

                break

            elif p_or_f == "next":

                continue

            else:
                
                pass
                    
    # 검색결과 Cleanmatch에 아예 없을 경우 search_results['matchCandidates']의 리스트 길이가 0이라 indexError 발생
    except IndexError:

        duns = '000'
        companyname = search_list[c_num]['회사명']
        matchscore = 0
        address = ''
        regi_number = ''

    finally:        

        datablock_search[c_num] = {
            '구분': c_num,
            '회사명': companyname,
            '주소': address,
            '등록번호': regi_number,
            'score': matchscore,
            'duns': duns,
        }

에플
{'법인1': {'구분': '법인1', '회사명': '삼성전자(주)', '주소': {'addressCountry': {'isoAlpha2Code': 'KR', 'name': 'Republic Of Korea'}, 'addressLocality': {'name': '수원시'}, 'addressRegion': {'name': '경기도', 'abbreviatedName': None}, 'postalCode': '16677', 'postalCodeExtension': None, 'streetAddress': {'line1': '경기도 수원시 영통구 삼성로129(매탄동)', 'line2': None}}, '등록번호': [{'registrationNumber': '1248100998', 'typeDescription': 'Commercial Registry Business Number (KR)', 'typeDnBCode': 1387}], 'score': 7, 'duns': '687967554'}, '법인2': {'구분': '법인2', '회사명': '(주)에플', '주소': {'addressCountry': {'isoAlpha2Code': 'KR', 'name': 'Republic Of Korea'}, 'addressLocality': {'name': '전주시'}, 'addressRegion': {'name': '전라북도', 'abbreviatedName': None}, 'postalCode': '54961', 'postalCodeExtension': None, 'streetAddress': {'line1': '전라북도 전주시 완산구 척동3길5-6, 301호(효자동3가)', 'line2': None}}, '등록번호': [{'registrationNumber': '3358601821', 'typeDescription': 'Commercial Registry Business Number (KR)', 'typeDnBCode': 1387}], 'score': 5, 'duns

TypeError: 'NoneType' object is not subscriptable

In [55]:
datablock_search = {}
url = cleanmatch_url

headers = {
    'Authorization': 'Bearer ' + accesstoken['accessToken'],
    'Content-Type': 'application/json;charset=utf-8'
}

body = {
    'registrationNumber': search_list[c_num]['등록번호'],
    'name': search_list[c_num]['회사명'],
    'streetAddressLine1': search_list[c_num]['주소'],
    'countryISOAlpha2Code': search_list[c_num]['소재국가'],
    'candidateMaximumQuantity': search_param['후보갯수'],
    'confidenceLowerLevelThresholdValue': search_param['기준점'],
}

# 한국어 검색은 언어 parameter 수정 필요 - 한국어로(디폴트:en-US)
if body['countryISOAlpha2Code']=='KR':
    body['inLanguage'] = 'ko-hang-KR'

res = requests.get(url, headers=headers, params=body)
search_results = res.json()


{'transactionDetail': {'transactionID': 'rrt-0d431baa8887d59f2-b-se-17476-1324212-5',
  'transactionTimestamp': '2023-08-25T06:16:21.948Z',
  'inLanguage': 'en-US',
  'serviceVersion': '1'},
 'inquiryDetail': {'registrationNumber': '',
  'name': 'Hitachi Data Systems Pte Ltd',
  'address': {'countryISOAlpha2Code': 'SG',
   'streetAddressLine': {'line1': '18 Cross Street, #10-08 China Square Central Singapore'}},
  'candidateMaximumQuantity': 5,
  'confidenceLowerLevelThresholdValue': 5},
 'candidatesMatchedQuantity': 5,
 'matchDataCriteria': 'Name and Address Lookup',
 'matchCandidates': [{'displaySequence': 1,
   'organization': {'duns': '595174814',
    'websiteAddress': [],
    'dunsControlStatus': {'operatingStatus': {'description': 'Active',
      'dnbCode': 9074},
     'isMailUndeliverable': True},
    'primaryName': 'HITACHI VANTARA PTE. LIMITED',
    'tradeStyleNames': [],
    'telephone': [],
    'primaryAddress': {'addressCountry': {'isoAlpha2Code': 'SG',
      'name': 'Singa

In [56]:
len(search_results['matchCandidates'])

5

In [57]:
test_result = {}

for i in range(len(search_results['matchCandidates'])):
    
    test_sequence = search_results['matchCandidates'][i]
    
    duns = test_sequence['organization']['duns']
    companyname = test_sequence['organization']['primaryName']
    address = test_sequence['organization']['primaryAddress']
    regi_number = test_sequence['organization']['registrationNumbers']
    matchscore = test_sequence['matchQualityInformation']['confidenceCode']
    
    test_result[i] = {
            '구분': c_num,
            '회사명': companyname,
            '주소': address,
            '등록번호': regi_number,
            'score': matchscore,
            'duns': duns,
        }
    
test_result

{0: {'구분': '법인3',
  '회사명': 'HITACHI VANTARA PTE. LIMITED',
  '주소': {'addressCountry': {'isoAlpha2Code': 'SG', 'name': 'Singapore'},
   'addressLocality': {'name': 'Singapore'},
   'addressRegion': {},
   'postalCode': '199555',
   'postalCodeExtension': None,
   'streetAddress': {'line1': '300 Beach Road #19-01/07 The Concourse',
    'line2': None}},
  '등록번호': [{'registrationNumber': '198900599R',
    'typeDescription': 'Registration File Number (SG)',
    'typeDnBCode': 1386}],
  'score': 7,
  'duns': '595174814'},
 1: {'구분': '법인3',
  '회사명': 'DR DATA SYSTEMS',
  '주소': {'addressCountry': {'isoAlpha2Code': 'SG', 'name': 'Singapore'},
   'addressLocality': {'name': 'Singapore'},
   'addressRegion': {},
   'postalCode': '369144',
   'postalCodeExtension': None,
   'streetAddress': {'line1': '54 Jalan Gembira', 'line2': None}},
  '등록번호': [{'registrationNumber': '39227500K',
    'typeDescription': 'Registration File Number (SG)',
    'typeDnBCode': 1386}],
  'score': 7,
  'duns': '595530242

In [58]:
search_list[c_num]

{'등록번호': '',
 '회사명': 'Hitachi Data Systems Pte Ltd',
 '소재국가': 'SG',
 '주소': '18 Cross Street, #10-08 China Square Central Singapore'}

In [59]:
data_results = {}

headers = {
    'Authorization': 'Bearer ' + accesstoken['accessToken'],
    'Content-Type': 'application/json;charset=utf-8'
}

# 토큰시간 만료되었으면 재인증, 아니면 진행
if datetime.datetime.now() >= accesstoken['end_time']:

    accesstoken = get_access_token()
    headers['Authorization'] = 'Bearer ' + accesstoken['accessToken']

dunsNumber = '595174814'

datablock_url = f'https://gate.nicednb.com/global/directplus/v1.0/enrich/dproduct/datablock/{dunsNumber}'

body = {
    'blockIDs': 'companyinfo_L2_v1'
}

res = requests.get(datablock_url, headers=headers, params=body)
search_results = res.json()

data_results[c_num] = search_results

In [60]:
data_results

{'법인3': {'transactionDetail': {'transactionID': 'rrt-0746dbf852a36d197-b-se-17266-2099078-4',
   'transactionTimestamp': '2023-08-25T07:08:13.653Z',
   'inLanguage': 'en-US'},
  'inquiryDetail': {'duns': '595174814', 'blockIDs': ['companyinfo_L2_v1']},
  'organization': {'duns': '595174814',
   'isFortune1000Listed': None,
   'isForbesLargestPrivateCompaniesListed': None,
   'industryCodes': [{'code': '423430',
     'description': 'Computer and Computer Peripheral Equipment and Software Merchant Wholesalers',
     'typeDescription': 'North American Industry Classification System 2022',
     'typeDnBCode': 37788,
     'priority': 1},
    {'code': '424120',
     'description': 'Stationery and Office Supplies Merchant Wholesalers',
     'typeDescription': 'North American Industry Classification System 2022',
     'typeDnBCode': 37788,
     'priority': 2},
    {'code': '811210',
     'description': 'Electronic and Precision Equipment Repair and Maintenance',
     'typeDescription': 'North 

In [ ]:
# .py코드 : 전체 반복

for c_num in search_list.keys():

    print(c_num)

    # 토큰시간 만료되었으면 재인증, 아니면 진행
    if datetime.datetime.now() >= accesstoken['end_time']:

        accesstoken = get_access_token()
        headers['Authorization'] = 'Bearer ' + accesstoken['accessToken']

    body = {
        'registrationNumber': search_list[c_num]['등록번호'],
        'name': search_list[c_num]['회사명'],
        'streetAddressLine1': search_list[c_num]['주소'],
        'countryISOAlpha2Code': search_list[c_num]['소재국가'],
        'candidateMaximumQuantity': search_param['후보갯수'],
        'confidenceLowerLevelThresholdValue': search_param['기준점'],
    }

    # 한국어 검색은 언어 parameter 수정 필요 - 한국어로(디폴트:en-US)
    if body['countryISOAlpha2Code']=='KR':
        body['inLanguage'] = 'ko-hang-KR'

    res = requests.get(url, headers=headers, params=body)
    search_results = res.json()

    # 일단은 1번으로 나오는 결과물을 뽑도록 해 놨으나, 정확도 보완차원에서 결과물 다 검토하긴 해야함
    try:

        best_results = search_results['matchCandidates'][0]

        duns = best_results['organization']['duns']
        companyname = best_results['organization']['primaryName']
        address = best_results['organization']['primaryAddress']
        regi_number = best_results['organization']['registrationNumbers']
        matchscore = best_results['matchQualityInformation']['confidenceCode']

    # 결과 없을 경우 search_results['matchCandidates']의 리스트 길이가 0이라 indexError 발생
    except IndexError:

        duns = '000'
        companyname = search_list[c_num]['회사명']
        matchscore = 0
        address = ''
        regi_number = ''

    finally:        

        datablock_search[c_num] = {
            '구분': c_num,
            '회사명': companyname,
            '주소': address,
            '등록번호': regi_number,
            'score': matchscore,
            'duns': duns,
        }